In [1]:
import dev.biserman.planet.language.Language
import dev.biserman.planet.language.Segment
import dev.biserman.planet.language.SyllableConstructor
import dev.biserman.planet.language.InventoryTransformation
import dev.biserman.planet.language.Manner
import dev.biserman.planet.language.SegmentType


In [62]:
import dev.biserman.planet.language.Glide
import dev.biserman.planet.language.Place
import dev.biserman.planet.language.SegmentData
import dev.biserman.planet.utils.toWeightedBag
import kotlin.random.Random

SyllableConstructor.languageFile = """E:\Users\Winggar\source\repos\Planet\planet\english.json"""
SyllableConstructor.phonemeFile = """E:\Users\Winggar\source\repos\Planet\planet\phonemes.json"""

val basicTransformation: InventoryTransformation = { inventory ->
    inventory.plus("ptk s mn ljw".mapNotNull { SyllableConstructor.segments[it.toString()] })
}

fun (Set<Segment>).addSet(
    from: (SegmentData) -> Boolean,
    to: (SegmentData) -> SegmentData,
    condition: Boolean = true
): Set<Segment> {
    if (!condition) return this

    val new = this
        .filter { from(it.data) }
        .map { to(it.data) }
    val matching = SyllableConstructor.segments.values.filter { it.data in new }
    return this.plus(matching)
}

val random = Random(System.currentTimeMillis())

val placeWeights = mapOf(
    Place.BILABIAL to 45,
    Place.LABIODENTAL to 35,
    Place.DENTAL to 5,
    Place.ALVEOLAR to 50,
    Place.POSTALVEOLAR to 35,
    Place.PALATAL to 25,
    Place.VELAR to 45,
    Place.UVULAR to 5,
    Place.GLOTTAL to 50,
)

val inversePlaceWeights = (placeWeights.values.max()).let { maxWeight ->
    placeWeights.mapValues { (_, weight) -> maxWeight - weight + 1 }
}

//val affricates = mapOf(
//    'ɸ' to Pair('p', 10),
//    'β' to Pair('b', 10),
//    'f' to Pair('p', 50),
//    'v' to Pair('b', 50),
//    'θ' to Pair('t', 10),
//    'ð' to Pair('d', 10),
//    's' to Pair('t', 100),
//    'z' to Pair('d', 100),
//    'ʃ' to Pair('t', 250),
//    'ʒ' to Pair('d', 250),
//    'x' to Pair('k', 100)
//)

fun center(x: Double) = 4 * (x - 0.5).pow(3) + 0.5

val inventoryTransformations = listOf<Pair<InventoryTransformation, Int>>(
    { inventory: Set<Segment> -> // add voicing
        val chance = random.nextDouble()
        val newInventory = inventory.let {
            if (chance <= 0.9) {
                inventory.addSet(
                    from = { it.manner == Manner.PLOSIVE },
                    to = { it.copy(voiced = true) },
                    condition = inventory.all { it.data.manner != Manner.PLOSIVE || it.data.isAspirated != true } || random.nextDouble() <= 0.25)
            } else it
        }.let {
            if (chance <= 0.4 || chance > 0.9) {
                inventory.addSet(
                    from = { it.manner == Manner.FRICATIVE },
                    to = { it.copy(voiced = true) },
                    condition = inventory.all { it.data.manner != Manner.PLOSIVE || it.data.isAspirated != true } || random.nextDouble() <= 0.25)
            } else it
        }
        newInventory
    } to 600,
    { inventory: Set<Segment> -> // add aspiration
        inventory.addSet(
            from = { it.manner == Manner.PLOSIVE || it.manner == Manner.FRICATIVE },
            to = { it.copy(isAspirated = true, voiced = null) },
            condition = inventory.all { it.data.manner != Manner.PLOSIVE || it.data.isAspirated != true } || random.nextDouble() <= 0.25)
    } to 400,
    { inventory: Set<Segment> -> // add ejectives
        inventory.addSet(
            from = { it.manner == Manner.PLOSIVE || it.manner == Manner.FRICATIVE },
            to = { it.copy(isEjective = true, voiced = null) })
    } to 100,

    { inventory: Set<Segment> -> // add plosives from fricatives
        inventory.addSet(
            from = { it.manner == Manner.FRICATIVE },
            to = { it.copy(manner = Manner.PLOSIVE) })
    } to 300,
    { inventory: Set<Segment> -> // add fricatives from plosives
        inventory.addSet(
            from = { it.manner == Manner.PLOSIVE },
            to = { it.copy(manner = Manner.FRICATIVE) })
    } to 300,

    { inventory: Set<Segment> -> // add rhotic
        inventory.plus(SyllableConstructor.segments["ɹ"]!!)
    } to 500,
    { inventory: Set<Segment> -> // add implosives
        inventory.addSet(
            from = { it.manner == Manner.PLOSIVE || it.manner == Manner.FRICATIVE },
            to = { it.copy(manner = Manner.IMPLOSIVE) })
    } to 100,
    { inventory: Set<Segment> -> // add clicks
        inventory.addSet(
            from = { it.manner == Manner.PLOSIVE || it.manner == Manner.FRICATIVE },
            to = { it.copy(manner = Manner.CLICK, voiced = null) })
    } to 100,
    { inventory: Set<Segment> -> // add nasals
        inventory.addSet(
            from = { it.manner == Manner.PLOSIVE || it.manner == Manner.FRICATIVE },
            to = { it.copy(manner = Manner.NASAL, voiced = null) })
    } to 250,
    { inventory: Set<Segment> -> // remove randomly
        inventory.filter { random.nextDouble() <= center(it.prevalence).pow(0.33) }.toSet()
    } to 500,

    affricate@{ inventory: Set<Segment> -> // add affricates
        val fricative =
            inventory.filter { it.data.manner == Manner.FRICATIVE && it.data.place != Place.GLOTTAL }
                .randomOrNull(random) ?: return@affricate inventory
        val glide = Glide.from(fricative.data)
        val affricate = if (random.nextDouble() <= 0.75) {
            val place = when (fricative.data.place) {
                Place.LABIODENTAL -> Place.BILABIAL
                Place.POSTALVEOLAR -> Place.ALVEOLAR
                Place.PALATAL -> Place.ALVEOLAR
                Place.DENTAL -> Place.ALVEOLAR
                else -> fricative.data.place
            }
            fricative.copyData { it.copy(place = place, manner = Manner.PLOSIVE, consonantGlide = glide) }
        } else {
            val plosive =
                inventory
                    .filter {
                        it.data.manner == Manner.PLOSIVE &&
                                it.data.place != Place.GLOTTAL &&
                                it.data.isAspirated == false &&
                                it.data.isEjective == false &&
                                it.data.voiced == fricative.data.voiced
                    }
                    .randomOrNull(random) ?: return@affricate inventory
            plosive.copyData { it.copy(consonantGlide = glide) }
        }
        inventory.plus(affricate)
    } to 300,

    glideColoredSet@{ inventory: Set<Segment> -> // add a glide-colored set
        val chance = random.nextDouble()
        val glide = SyllableConstructor.segments.values
            .filter { it.data.manner == Manner.SEMIVOWEL || it.data.manner == Manner.LIQUID }
            .filter { it in inventory }
            .randomOrNull(random) ?: return@glideColoredSet inventory

        val newInventory = inventory.let {
            if (chance <= 0.67) {
                it.plus(it.filter { it.data.manner == Manner.FRICATIVE }
                    .map { it.copyData { it.copy(consonantGlide = Glide.from(glide.data)) } })
            } else it
        }.let {
            if (chance <= 0.33 || chance > 0.67) {
                it.plus(it.filter { it.data.manner == Manner.PLOSIVE }
                    .filter { it.data.place != Place.GLOTTAL }
                    .map { it.copyData { it.copy(consonantGlide = Glide.from(glide.data)) } })
            } else it
        }

        newInventory
    } to 200,

    glideColoredSingle@{ inventory: Set<Segment> ->
        val chance = random.nextDouble()
        val glide = SyllableConstructor.segments.values
            .filter { it.data.manner == Manner.SEMIVOWEL || it.data.manner == Manner.LIQUID }
            .filter { it in inventory }
            .randomOrNull(random) ?: return@glideColoredSingle inventory

        val plosive = SyllableConstructor.segments.values
            .filter { it.data.manner == Manner.PLOSIVE }
            .filter { it.data.place != Place.GLOTTAL }
            .filter { it in inventory }
            .randomOrNull(random) ?: return@glideColoredSingle inventory

        inventory.plus(plosive.copyData { it.copy(consonantGlide = Glide.from(glide.data)) })
    } to 300,

    { inventory: Set<Segment> -> // tʃ, dʒ
        if (inventory.any { it.data.place == Place.ALVEOLAR && it.data.manner == Manner.PLOSIVE }) {
            val tʃ = SyllableConstructor.segments["t"]!!.copyData {
                it.copy(
                    consonantGlide = Glide(
                        Place.POSTALVEOLAR,
                        Manner.FRICATIVE
                    )
                )
            }
            val dʒ = SyllableConstructor.segments["d"]!!.copyData {
                it.copy(
                    consonantGlide = Glide(
                        Place.POSTALVEOLAR,
                        Manner.FRICATIVE
                    )
                )
            }

            if (inventory.any { it.data.place == Place.ALVEOLAR && it.data.manner == Manner.PLOSIVE && it.data.voiced == true }) {
                inventory.plus(listOf(tʃ, dʒ))
            } else {
                inventory.plus(tʃ)
            }
        } else {
            inventory
        }
    } to 300,
).plus(placeWeights.map { (place, weight) ->
    { inventory: Set<Segment> ->
        inventory.addSet(
            from = { it.type == SegmentType.CONSONANT },
            to = { it.copy(place = place) })

    } to weight * 3
}).plus(inversePlaceWeights.map { (place, weight) ->
    { inventory: Set<Segment> ->
        inventory.filter { it.data.place != place }.toSet()
    } to weight
})

val bag = inventoryTransformations.toWeightedBag(random) { it.second }

for (_1 in 0..10) {
    val testLanguage = basicTransformation(setOf()).let {
        (1..15).fold(it) { acc, _ -> bag.grab()!!.first.invoke(acc) }
    }.sortedBy { SyllableConstructor.segments.keys.indexOf(it.symbol) }
    println("${testLanguage.size} phonemes: ${testLanguage.map { it.display }}")
}


16 phonemes: [m, n, ŋ, p, t, ts, tʃ, tj, k, kɹ, s, z, j, w, ɹ, l]
20 phonemes: [m, n, ŋ, p, pj, t, ts, tʃ, k, b, d, dʒ, ɡ, s, ʃ, z, ʒ, j, w, l]
21 phonemes: [m, n, ŋ, p, pl, t, tʃ, k, b, bz, d, dʒ, ɡ, s, z, ʔ, h, j, w, ɹ, l]
15 phonemes: [m, n, p, t, k, d, s, ʃ, z, ʒ, ʔ, h, j, w, l]
12 phonemes: [m, n, p, pw, t, tl, tʃ, k, dʒ, ɡ, w, l]
30 phonemes: [m, n, p, pl, t, ts, tl, k, kl, kj, b, bl, d, dl, ɡ, ɡl, f, fj, fl, s, sw, sj, sl, v, z, ʔ, h, j, w, l]
16 phonemes: [m, n, p, t, ts, tʃ, k, b, d, dʒ, ɡ, s, j, w, ɹ, l]
15 phonemes: [m, n, ŋ, p, t, tʃ, k, kw, b, d, ɡ, s, ʒ, w, l]
19 phonemes: [m, n, ŋ, p, t, ts, k, b, d, ɡ, ɡw, f, s, v, z, j, w, ɹ, l]
21 phonemes: [m, n, pj, t, tj, tʃ, k, kj, bj, dj, dʒ, dl, d, ɡ, ɡj, s, sj, j, w, ɹ, l]
12 phonemes: [m, n, p, t, tʃ, k, b, d, dʒ, dw, ɡ, w]


In [3]:
SyllableConstructor.segments.entries.map {
    println("${it.key} - ${it.value}")
}

m - Segment(symbol=m, data=SegmentData(type=CONSONANT, place=BILABIAL, manner=NASAL, voiced=null, isAspirated=false, isEjective=false, height=null, depth=null, rounded=null, consonantGlide=null, onGlide=null, offGlide=null), prevalence=0.97)
n - Segment(symbol=n, data=SegmentData(type=CONSONANT, place=ALVEOLAR, manner=NASAL, voiced=null, isAspirated=false, isEjective=false, height=null, depth=null, rounded=null, consonantGlide=null, onGlide=null, offGlide=null), prevalence=0.97)
ŋ - Segment(symbol=ŋ, data=SegmentData(type=CONSONANT, place=VELAR, manner=NASAL, voiced=null, isAspirated=false, isEjective=false, height=null, depth=null, rounded=null, consonantGlide=null, onGlide=null, offGlide=null), prevalence=0.69)
p - Segment(symbol=p, data=SegmentData(type=CONSONANT, place=BILABIAL, manner=PLOSIVE, voiced=false, isAspirated=false, isEjective=false, height=null, depth=null, rounded=null, consonantGlide=null, onGlide=null, offGlide=null), prevalence=0.95)
t - Segment(symbol=t, data=Segme

[kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit, kotlin.Unit]

In [4]:
SyllableConstructor.glideMap

{(Glide(place=LABIODENTAL, manner=FRICATIVE), false)=f, (Glide(place=DENTAL, manner=FRICATIVE), false)=θ, (Glide(place=ALVEOLAR, manner=FRICATIVE), false)=s, (Glide(place=POSTALVEOLAR, manner=FRICATIVE), false)=ʃ, (Glide(place=LABIODENTAL, manner=FRICATIVE), true)=v, (Glide(place=ALVEOLAR, manner=FRICATIVE), true)=z, (Glide(place=POSTALVEOLAR, manner=FRICATIVE), true)=ʒ, (Glide(place=PALATAL, manner=SEMIVOWEL), null)=j, (Glide(place=LABIOVELAR, manner=SEMIVOWEL), null)=w, (Glide(place=RETROFLEX, manner=LIQUID), null)=ɹ, (Glide(place=ALVEOLAR, manner=LIQUID), null)=l}

In [5]:
fun generateVowels(): List<Segment> {
    return SyllableConstructor.segments.values.filter { it.data.type == SegmentType.VOWEL }
        .filter { random.nextDouble() <= it.data.prevalence ?: 0.0 }
}

for (_1 in 0..10) {
    print(generateVowels().map { it.display })
}

org.jetbrains.kotlinx.jupyter.exceptions.ReplCompilerException: at Cell In[5], line 3, column 50: Unresolved reference: prevalence